## Import necessary packages

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import RidgeCV

import warnings
warnings.filterwarnings("ignore")

## Read dataset

In [2]:
df = pd.read_csv('data/StockX-Data-Contest-2019-3.csv')
df.head()

,Order Date,Brand,Sneaker Name,Sale Price,Retail Price,Release Date,Shoe Size,Buyer Region
0,9/1/17,Yeezy,Adidas-Yeezy-Boost-350-Low-V2-Beluga,"$1,097",$220,9/24/16,11.0,California
1,9/1/17,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Copper,$685,$220,11/23/16,11.0,California
2,9/1/17,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Green,$690,$220,11/23/16,11.0,California
3,9/1/17,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red,"$1,075",$220,11/23/16,11.5,Kentucky
4,9/1/17,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red-2017,$828,$220,2/11/17,11.0,Rhode Island


## Data Cleaning

#### With a glimpse of the values in each columns in the dataset, we can notice that variables Order Date, Release Date, Sale Price, and Retail Price need to go through some data cleaning and transformation.
#### For the categorical variables including Brand, Sneaker Name, and Buyer Region, I will transform them with one-hot matrices so that they can be interpreted as numerical values.
### Specifically how:
#### For Release Date and Order Date, I will calculate the difference between the two dates and store them as 'diff_days' as a new column.
#### For Sale Price and Retail Price, I will transform the data values so that they will not have a dollar sign and comma.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99956 entries, 0 to 99955
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Order Date    99956 non-null  object 
 1   Brand         99956 non-null  object 
 2   Sneaker Name  99956 non-null  object 
 3   Sale Price    99956 non-null  object 
 4   Retail Price  99956 non-null  object 
 5   Release Date  99956 non-null  object 
 6   Shoe Size     99956 non-null  float64
 7   Buyer Region  99956 non-null  object 
dtypes: float64(1), object(7)
memory usage: 6.1+ MB


## Check for null or empty values
### In this dataset, we don't have any empty or null values

In [4]:
df.isnull().sum()

Order Date      0
Brand           0
Sneaker Name    0
Sale Price      0
Retail Price    0
Release Date    0
Shoe Size       0
Buyer Region    0
dtype: int64

In [5]:
temp0 = []
temp1 = []
for price in df['Sale Price']:
    temp0.append(price[1:])
for num in temp0:
    temp_v = 0
    if ',' in num:
        temp_v = int(num[-1]) + int(num[-2]) * 10 + int(num[-3]) * 100 + int(num[-5]) * 1000
        temp1.append(temp_v)
    else:
        temp1.append(int(num))
df['Sale Price'] = temp1

temp1 = []
for price in df['Retail Price']:
    temp1.append(int(price[1:]))
df['Retail Price'] = temp1

df.head()

,Order Date,Brand,Sneaker Name,Sale Price,Retail Price,Release Date,Shoe Size,Buyer Region
0,9/1/17,Yeezy,Adidas-Yeezy-Boost-350-Low-V2-Beluga,1097,220,9/24/16,11.0,California
1,9/1/17,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Copper,685,220,11/23/16,11.0,California
2,9/1/17,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Green,690,220,11/23/16,11.0,California
3,9/1/17,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red,1075,220,11/23/16,11.5,Kentucky
4,9/1/17,Yeezy,Adidas-Yeezy-Boost-350-V2-Core-Black-Red-2017,828,220,2/11/17,11.0,Rhode Island


In [6]:
onehot0 = pd.get_dummies(df['Brand'])
onehot1 = pd.get_dummies(df['Sneaker Name'])
onehot2 = pd.get_dummies(df['Buyer Region'])
df = df.drop(columns='Brand')
df = df.drop(columns='Sneaker Name')
df = df.drop(columns='Buyer Region')
df = df.join(onehot0)
df = df.join(onehot1)
df = df.join(onehot2)

df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Release Date'] = pd.to_datetime(df['Release Date'])

df['diff_days'] = df['Order Date'] - df['Release Date']
df = df.drop(columns='Order Date')
df = df.drop(columns='Release Date')

df['diff_days'] /= np.timedelta64(1,'D')

df['increase_perday'] = (df['Sale Price'] - df['Retail Price']) / df['diff_days'] # I included this as an additional feature, but it turns out not having a high correlation with Sale Price

df

,Sale Price,Retail Price,Shoe Size,Yeezy,Off-White,Adidas-Yeezy-Boost-350-Low-Moonrock,Adidas-Yeezy-Boost-350-Low-Oxford-Tan,Adidas-Yeezy-Boost-350-Low-Pirate-Black-2015,Adidas-Yeezy-Boost-350-Low-Pirate-Black-2016,Adidas-Yeezy-Boost-350-Low-Turtledove,...,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,diff_days,increase_perday
0,1097,220,11.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,342.0,2.564327
1,685,220,11.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,282.0,1.648936
2,690,220,11.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,282.0,1.666667
3,1075,220,11.5,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,282.0,3.031915
4,828,220,11.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,202.0,3.009901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99951,565,220,8.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,49.0,7.040816
99952,598,220,8.5,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,49.0,7.714286
99953,605,220,5.5,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,49.0,7.857143
99954,650,220,11.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,49.0,8.775510


## Getting the correlations of each of the variables

In [7]:
df.corr()['Sale Price'].sort_values(ascending=False)

Sale Price                                           1.000000
Off-White                                            0.545128
Air-Jordan-1-Retro-High-Off-White-Chicago            0.450942
Air-Jordan-1-Retro-High-Off-White-University-Blue    0.348208
Air-Jordan-1-Retro-High-Off-White-White              0.302024
                                                       ...   
Adidas-Yeezy-Boost-350-V2-Sesame                    -0.172974
Adidas-Yeezy-Boost-350-V2-Cream-White               -0.195213
adidas-Yeezy-Boost-350-V2-Butter                    -0.248195
Retail Price                                        -0.361550
 Yeezy                                              -0.545128
Name: Sale Price, Length: 108, dtype: float64

## Only the correlations with absolute values greater than 0.1 will be considered as relevant features, others will be dropped

In [8]:
cor = df.corr()
cor_target = abs(cor['Sale Price'])
relevant_features = cor_target[cor_target>0.1]
relevant_features
df = df[relevant_features.index]
sns.pairplot(df, height = 1.2, aspect=1.5)

### The reason why I didn't take the Log value of the Sale Price to reduce its skewness is because the correlation between variables are already very low as shown by the graph. Also, I included taking the Log of the Sale Price at the beginning and run the same model, but it largely lowered R squared for both train and test dataset.

## Use pipeline to scale the data and then run RidgeCV

### Due to the complexity of this model, I decided to use degree=2 as my optimal degree.

In [16]:
x = df[relevant_features.index]
x = x.drop(columns='Sale Price')
y = df['Sale Price']
X_train, X_test, y_train, y_test = train_test_split(x,y,shuffle=True,test_size=0.3)
model = make_pipeline(StandardScaler(), PolynomialFeatures(degree=2), RidgeCV(cv=10, alphas = [1e-8, 1e-1, 1, 10, 12, 15]))
model.fit(X_train,y_train)
print('Optimal alpha: ', model.steps[2][1].alpha_)

print('Train R-squared:', model.score(X_train, y_train))
print(f"Train Adjusted R^2: {1 - (1-model.score(X_train, y_train))*(len(y_train)-1)/(len(y_train)-x.shape[1]-1)}")
print('Test R-squared:', model.score(X_test, y_test))
print(f"Test Adjusted R^2: {1 - (1-model.score(X_test, y_test))*(len(y_test)-1)/(len(y_test)-x.shape[1]-1)}")

predict = model.predict(X_test)
new_df = X_test
new_df["real_price"] = y_test
new_df["predict_price"] = predict
scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print('Average MSE: ', np.mean(-scores))
print('MSE: ', mean_squared_error(y_test, predict))
new_df

Optimal alpha:  0.1
Train R-squared: 0.7471365987319043
Train Adjusted R^2: 0.747071530236939
Test R-squared: 0.755625880158402
Test Adjusted R^2: 0.7554790991200562
Average MSE:  16529.83933201362
MSE:  16132.961242811047


,Retail Price,Yeezy,Off-White,Adidas-Yeezy-Boost-350-Low-Turtledove,Adidas-Yeezy-Boost-350-V2-Core-Black-Red,Adidas-Yeezy-Boost-350-V2-Core-Black-Red-2017,Adidas-Yeezy-Boost-350-V2-Cream-White,Adidas-Yeezy-Boost-350-V2-Sesame,Adidas-Yeezy-Boost-350-V2-Zebra,Air-Jordan-1-Retro-High-Off-White-Chicago,Air-Jordan-1-Retro-High-Off-White-University-Blue,Air-Jordan-1-Retro-High-Off-White-White,Nike-Air-Presto-Off-White,Nike-Air-Presto-Off-White-Black-2018,Nike-Air-Presto-Off-White-White-2018,Nike-Air-VaporMax-Off-White,adidas-Yeezy-Boost-350-V2-Butter,adidas-Yeezy-Boost-350-V2-Static,real_price,predict_price
31407,190,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,905,851.676166
39222,220,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,263,269.584739
88753,220,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,275,287.396123
15783,160,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,690,497.718549
74568,190,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1000,851.676166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20156,220,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,460,416.865068
28972,250,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,569,607.909840
20190,220,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,389,416.865068
23106,220,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,399,416.865068


### The R^2, adjusted R^2, and MSE value are close between training set and testing set, therefore the model fits well with the data

## Model coefficients and intercept

In [20]:
print(model.steps[2][1].intercept_)
print(model.steps[2][1].coef_)

695.3777906697002
[ 0.00000000e+00 -3.13241255e+02  1.51686244e+01 -1.51686244e+01
  1.35630414e-02  6.26644255e-02  2.57779729e-01 -7.14038652e+00
 -3.57100521e+00 -5.99024377e+00  9.16214878e-01  4.73264595e+00
  2.76613973e-01  5.94152609e-01  1.80196291e+00  1.61708228e+00
 -1.17892540e-01 -1.06704888e+01 -2.72678350e+00  2.52196478e+01
 -1.62872119e+02  1.62872119e+02  8.37194038e+00  1.68414243e+01
  3.11246921e+01  9.64924399e+01  7.42136979e+01  1.02859487e+02
  2.64287464e+01  6.52844095e+01  1.74681498e+01  2.33423423e+01
  3.98790574e+01  4.05201041e+01  2.26766486e+01  1.07302265e+02
  7.00084703e+01 -1.50669768e+01  1.50669767e+01 -3.97225808e-01
 -7.75326872e-01 -1.34177575e+00 -9.25494022e+00 -5.88545906e+00
 -8.82516250e+00 -2.79057380e+00 -1.09651599e+01 -1.30200592e+00
 -2.14427919e+00 -5.00621503e+00 -4.72175505e+00 -9.17294757e-01
 -1.20438345e+01 -5.14004574e+00 -1.50669767e+01  3.97225812e-01
  7.75326856e-01  1.34177577e+00  9.25494016e+00  5.88545903e+00
  8.825

## Model Formula

$\hat{y} = -3.137Xretail + -5.084Xyezzy + 5.084Xow + ... X_n + 702.983$

#### Bias refers to the error due to the model’s simplistic assumptions in fitting the data. In my model it is the difference between the predicted sale price for a specific shoe and the real sale price of that specific show. Variance refers to the error due to the complex model trying to fit the data. As my training set R^2 and testing set R^2 are close, the variance is relatively small.

### 